In [7]:
import pandas as pd
import numpy as np
import random
import sklearn
from sklearn.model_selection import train_test_split
#from sklearn.linear_model import LogisticRegression
from deap import base, creator, tools, algorithms
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
#from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
import pickle
pickle_name="GA_input_features.pickle"
with open(pickle_name, 'rb') as handle:
    GA_input_features = pickle.load(handle)
import matplotlib.pyplot as plt

In [10]:
from  ML_initial_test import ML
train='./small/session-1-DeviceGroup-1.csv'
#train='./Aalto_BIG_train_IoTDevID.csv'
vali='./small/session-2-DeviceGroup-2.csv'
test='./small/session-1-DeviceGroup-2.csv'
fold=1
#df=ML(GA_input_features,train,vali,test,fold)

In [38]:
from  ML_initial_test import ML
#ml_list={"BaggingClassifier":BaggingClassifier()}
ml_list={"DecisionTreeClassifier":DecisionTreeClassifier()}
model=DecisionTreeClassifier()
fold=10
df=ML(GA_input_features,train,vali,test,fold,ml_list)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:07<00:00,  7.13s/it]

    ML                        Vali-ACC    Vali-F1    Vali-STD    Test-ACC    Test-F1    Test-STD    Total-TIME
--  ----------------------  ----------  ---------  ----------  ----------  ---------  ----------  ------------
 0  DecisionTreeClassifier      0.9899     0.7443      0.0496      0.5649     0.5502      0.0057        7.1308


In [39]:
def FeatureMaker(f):
    selected=[]
    for i,ii in enumerate(f):
        if ii:
            selected.append(GA_input_features[i])
    return selected

In [40]:
def ReadCSV(features,train,vali,test):
    df = pd.read_csv(train,usecols=features)#,header=None )
    X_train =df[df.columns[0:-1]]
    #X_train=np.array(X_train)
    df[df.columns[-1]] = df[df.columns[-1]].astype('category')
    y_train=df[df.columns[-1]].cat.codes  
    
    df = pd.read_csv(vali,usecols=features)#,header=None )
    X_vali =df[df.columns[0:-1]]
    #X_test=np.array(X_test)
    df[df.columns[-1]] = df[df.columns[-1]].astype('category')
    y_vali=df[df.columns[-1]].cat.codes  
    
    df = pd.read_csv(test,usecols=features)#,header=None )
    X_test =df[df.columns[0:-1]]
    #X_test=np.array(X_test)
    df[df.columns[-1]] = df[df.columns[-1]].astype('category')
    y_test=df[df.columns[-1]].cat.codes  
    return X_train, y_train,  X_vali, y_vali, X_test, y_test

In [41]:
file_list={"DD1":['./small/session-1-DeviceGroup-1.csv','./small/session-2-DeviceGroup-2.csv','./small/session-1-DeviceGroup-2.csv'],
"DD1T":['./small/session-2-DeviceGroup-2.csv','./small/session-1-DeviceGroup-1.csv','./small/session-2-DeviceGroup-1.csv'],
"DD2":['./small/session-2-DeviceGroup-1.csv','./small/session-1-DeviceGroup-2.csv','./small/session-2-DeviceGroup-2.csv'],
"DD2T":['./small/session-1-DeviceGroup-2.csv','./small/session-2-DeviceGroup-1.csv','./small/session-1-DeviceGroup-1.csv']}

In [48]:
import warnings
warnings.filterwarnings('ignore')

for i in file_list:
    features=GA_input_features
    train=file_list[i][0]
    vali=file_list[i][1]
    test=file_list[i][2]
    X_train, y_train, X_test, y_test, X_final, y_final=ReadCSV(features,train,vali,test)
    break

In [49]:
# Ağırlığı 1 olan bir fitness fonksiyonu oluşturun. 
# Eğer maksimize edilen bir fonksiyonsa ağırlık pozitif olmalıdır, 
#aksi takdirde minimizasyon için negatif olmalıdır. Burada doğruluğu maksimize etmek istiyoruz.
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
# Fitness fonksiyonu ile tanımlanmış boş bir birey oluşturun.
# Birey, kromozomla eşanlamlıdır
creator.create("Individual", list, fitness=creator.FitnessMax)
# Kromozomun boyutunu belirtin. Burada boyut değişken sayısı olacaktır
ind_size = X_train.shape[1]
# Toolbox'ı tanımlayın. Toolbox, tüm tanımlanan operatörleri içeren bir konteynerdir
# Herhangi bir operatörü toolbox'a dahil etmek için kaydedilmelidir
toolbox = base.Toolbox()
# 1 veya 0'ı rastgele üretecek bir sayı üretici fonksiyonunu kaydedin.
# Bu, bir değişkenin bir bireyde mevcut olup olmadığını belirtir. 1: Var; 0: Yok
# Buradaki 1/0 değerleri alellerdir
toolbox.register("attrib_bin", random.randint, 0, 1)
# Yukarıda tanımlanan rastgele sayı üreteci ile bireylere değerler ekleyin
# Her bireyin boyutu değişken sayısı olacaktır
# Bireyleri toolbox'a kaydedin
# initRepeat argümanı bireyi başlatmak için kullanılır
toolbox.register(
    "individual", tools.initRepeat, creator.Individual, toolbox.attrib_bin, n=ind_size
)
# Yukarıda oluşturulan bireylerle bir populasyon tanımlayın
# Popülasyonu toolbox'a kaydedin
# initRepeat argümanı populasyonu başlatmak için kullanılır
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [50]:
ind1 = toolbox.individual()
print("Toplam değişken sayısı/her bir kromozom uzunluğu = ", ind_size)
print(ind1)

Toplam değişken sayısı/her bir kromozom uzunluğu =  222
[0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1]


In [51]:
def evaluate(individual):
    sum_features = np.sum(individual)
    if sum_features == 0:
        return 0.0
    else:
        sum_features = np.sum(individual)
        collector_train = []
        collector_test = []
        for k in range(0, len(individual)):
            if individual[k] == 1:
                collector_train.append(X_train.iloc[:, k])
                collector_test.append(X_test.iloc[:, k])
        X_train1 = np.transpose(np.array(collector_train))
        X_test1 = np.transpose(np.array(collector_test))
        np.nan_to_num(X_train1, copy=False)
        np.nan_to_num(y_train, copy=False)
        np.nan_to_num(X_test1, copy=False)
        np.nan_to_num(y_test, copy=False)
        clf = model.fit(X_train1, y_train)
        y_pred = clf.predict(X_test1)
        
        #f1 = accuracy_score(y_test['target'].tolist(),y_pred)
        f1 = f1_score(y_test.tolist(),y_pred, average='macro')
        #print("F1: ", f1_score(y_test['target'].tolist(),y_pred, average='macro'))
        #print("ACC: ", accuracy_score(y_test['target'].tolist(),y_pred))
        return (f1,)


In [52]:
# İki nokta çaprazlama
toolbox.register("mate", tools.cxTwoPoint)
# Bit flip mutasyonu. indpb argümanı her bir özniteliğin ters çevrilme olasılığıdır
toolbox.register("mutate", tools.mutFlipBit, indpb=0.01)
# Tournsize argümanıyla rastgele seçilen bireyler arasından en iyi bireyi seçin
toolbox.register("select", tools.selNSGA2)
#########################toolbox.register("select", tools.selTournament, tournsize=3)
# Yukarıda tanımlanan uygunluk fonksiyonunu toolbox'a kaydedin
toolbox.register("evaluate", evaluate)
# Algoritmanın çalışması sırasında gösterilecek istatistikleri tanımlayın.
# Her nesil için minimum, maksimum ve ortalama doğruluk seçtik
# Kararı, daha önce belirtildiği gibi maksimum doğruluk üzerine yapacağız
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("Mean", np.mean)
stats.register("Max", np.max)
stats.register("Min", np.min)


In [53]:
# İlk populasyonu, burada sadece gösterim için düşük bir populasyon boyutu (sadece 20) olarak tanımlıyoruz
pop = toolbox.population(n=20)
# HallOfFame argümanı, tüm nesiller boyunca en iyi bireyi kaydeder
# Bu, son değişken kombinasyonumuzu belirlemek için kullanılacaktır. 
# Sadece ilk en iyiyi alıyoruz. hof içinde k sayıda en iyi çözümü kaydetmek mümkündür
hof = tools.HallOfFame(1)
# Şimdi, algoritmayı DEAP'teki mevcut en basit yöntemle çalıştıralım (eaSimple).
# Argümanlar şu şekildedir:
# pop: Daha önce tanımlanan populasyon
# toolbox: tüm tanımlanan operatörleri içeren toolbox
# cxpb: İki bireyi çiftleştirmenin olasılığı.
# mutpb: Bir bireyi değiştirmenin olasılığı. Etkiyi göstermek için yüksek tutuyoruz
# ngen: Nesil sayısı.
pop, log = algorithms.eaSimple(
    pop, toolbox, cxpb=0.5, mutpb=0.3, ngen=50, halloffame=hof, stats=stats
)
# En iyi bireyi alın
best = hof.items[0]
# En iyi bireyi ve onun doğruluğunu yazdırın
print("En İyi Birey = ", best)
print("En İyi Uygunluk = ", best.fitness.values[0])

gen	nevals	Mean  	Max    	Min     
0  	20    	0.4752	0.53518	0.359633
1  	13    	0.476362	0.546284	0.310508
2  	15    	0.473859	0.546609	0.310508
3  	9     	0.476528	0.546618	0.310508
4  	15    	0.474771	0.546618	0.3594  
5  	12    	0.469356	0.546618	0.359239
6  	12    	0.465113	0.491062	0.358826
7  	8     	0.465324	0.535171	0.358826
8  	9     	0.465474	0.535171	0.358826
9  	18    	0.469924	0.546618	0.345978
10 	10    	0.471269	0.546618	0.320276
11 	10    	0.469665	0.546618	0.346335
12 	13    	0.471545	0.546609	0.346335
13 	10    	0.470056	0.528587	0.346335
14 	15    	0.466509	0.535171	0.339933
15 	14    	0.471287	0.540025	0.346335
16 	9     	0.47422 	0.540025	0.350846
17 	14    	0.465648	0.535171	0.350846
18 	6     	0.465729	0.535171	0.350846
19 	12    	0.4697  	0.53518 	0.350846
20 	16    	0.467251	0.53518 	0.350478
21 	16    	0.459979	0.53518 	0.351725
22 	17    	0.459773	0.53518 	0.347607
23 	9     	0.461098	0.53518 	0.356525
24 	10    	0.458534	0.53518 	0.356525
25 	8     	0.46210

In [29]:
X_train

array([[ 1.53890000e+04,  1.16259818e+09,  3.74200000e-03, ...,
        -9.99900000e+03, -9.99900000e+03,  0.00000000e+00],
       [-9.99900000e+03, -9.99900000e+03, -9.99900000e+03, ...,
         6.66700000e+03,  4.91530000e+04,  1.20000000e+01],
       [-9.99900000e+03, -9.99900000e+03, -9.99900000e+03, ...,
        -9.99900000e+03, -9.99900000e+03,  0.00000000e+00],
       ...,
       [-9.99900000e+03, -9.99900000e+03, -9.99900000e+03, ...,
         6.66700000e+03,  4.91540000e+04,  1.20000000e+01],
       [ 1.14110000e+04,  1.97767348e+09,  4.37800000e-03, ...,
        -9.99900000e+03, -9.99900000e+03,  0.00000000e+00],
       [-9.99900000e+03, -9.99900000e+03, -9.99900000e+03, ...,
         6.66700000e+03,  4.91550000e+04,  1.20000000e+01]])

In [22]:
best_collector = []
for k in range(0, 52):
    if best[k] == 1:
        best_collector.append(X_train.columns[k])
print("Seçilen değişken sayısı = ", len(best_collector))
print(best_collector)

Seçilen değişken sayısı =  33
['pck_size', 'EAPOL_version', 'EAPOL_type', 'EAPOL_len', 'IP_version', 'IP_ihl', 'IP_tos', 'IP_len', 'IP_flags', 'IP_DF', 'IP_ttl', 'IP_options', 'ICMP_type', 'ICMP_code', 'ICMP_seq', 'TCP_FIN', 'TCP_SYN', 'TCP_RST', 'TCP_window', 'UDP_len', 'DHCP_options', 'BOOTP_op', 'BOOTP_hlen', 'DNS_aa', 'DNS_rd', 'DNS_rcode', 'DNS_qdcount', 'DNS_ancount', 'DNS_arcount', 'sport_class', 'dport_class', 'payload_bytes', 'entropy']


In [23]:
X_train1_best = X_train[best_collector]
clf_best = model.fit(X_train1_best, y_train)
X_test1 = X_test[best_collector]
y_pred = clf_best.predict(X_test1)
accuracy = (y_test.tolist()==y_pred).sum() / y_test.count()
print(accuracy)
print("F1: ", f1_score(y_test.tolist(),y_pred, average='macro'))

0.7169424443278813
F1:  0.7303671015818862


In [24]:

final="Aalto_test_IoTDevID.csv"
df = pd.read_csv(final,usecols=cols)#,header=None )

X_final =df[df.columns[0:-1]]
#X_test=np.array(X_test)
df[df.columns[-1]] = df[df.columns[-1]].astype('category')
y_final=df[df.columns[-1]].cat.codes  


clf_best = model.fit(X_train1_best, y_train)
X_test1 = X_final[best_collector]
y_pred = clf_best.predict(X_test1)
accuracy = (y_final.tolist()==y_pred).sum() / y_final.count()
print(accuracy)
print("F1: ", f1_score(y_final.tolist(),y_pred, average='macro'))

0.7508528998595224
F1:  0.76958353433121


In [30]:
cols=feature=[#'IP_id',
#'ICMP_chksum',
#'ICMP_id',
#'TCP_seq',
#'TCP_ack',
 #'TCP_chksum',
# 'UDP_chksum',
# 'DNS_id',
# 'BOOTP_xid',
'sport_class',
'dport_class',
#'sport23',
#'dport23',
#'sport_bare',
# 'dport_bare',
# 'TCP_sport',
# 'TCP_dport',
# 'UDP_sport',
# 'UDP_dport',
 'pck_size',
 'Ether_type',
 'LLC_ssap',
 'LLC_ctrl',
 'EAPOL_version',
 'EAPOL_type',
 'EAPOL_len',
 'IP_version',
 'IP_ihl',
 'IP_tos',
 'IP_len',
 'IP_flags',
 'IP_DF',
 'IP_ttl',
 'IP_proto',
 'IP_options',
 'ICMP_type',
 'ICMP_code',
 'ICMP_seq',
 'TCP_dataofs',
 'TCP_flags',
 'TCP_FIN',
 'TCP_SYN',
 'TCP_RST',
 'TCP_PSH',
 'TCP_ACK',
 'TCP_window',
 'TCP_options',
 'UDP_len',
 'DHCP_options',
 'BOOTP_op',
 'BOOTP_htype',
 'BOOTP_hlen',
 'BOOTP_secs',
 'BOOTP_flags',
 'BOOTP_sname',
 'BOOTP_file',
 'BOOTP_options',
 'DNS_qr',
 'DNS_aa',
 'DNS_rd',
 'DNS_ra',
 'DNS_rcode',
 'DNS_qdcount',
 'DNS_ancount',
 'DNS_nscount',
 'DNS_arcount',
 'payload_bytes',    
 'entropy',
# 'Protocol',
#"MAC",
 'Label']




train='./UNSW_BIG_train_IoTDevID.csv'
final="UNSW_test_IoTDevID.csv"
df = pd.read_csv(final,usecols=cols)#,header=None )
X_final =df[df.columns[0:-1]]
#X_test=np.array(X_test)
df[df.columns[-1]] = df[df.columns[-1]].astype('category')
y_final=df[df.columns[-1]].cat.codes  

df = pd.read_csv(train,usecols=cols)#,header=None )
X_train =df[df.columns[0:-1]]
#X_train=np.array(X_train)
df[df.columns[-1]] = df[df.columns[-1]].astype('category')
y_train=df[df.columns[-1]].cat.codes  



results=[]
for i in range(10):
    model.fit(X_train,y_train)
    predictions = model.predict(X_final)
    results.append(f1_score(y_final,predictions,average= "macro"))
print ('%-30s %-30s' % ("MEAN","STD"))
print ('%-30s %-30s' % (np.mean(results),np.std(results)))

cols=feature=[#'IP_id',
#'ICMP_chksum',
#'ICMP_id',
#'TCP_seq',
#'TCP_ack',
 #'TCP_chksum',
# 'UDP_chksum',
# 'DNS_id',
# 'BOOTP_xid',
'sport_class',
'dport_class',
#'sport23',
#'dport23',
#'sport_bare',
# 'dport_bare',
# 'TCP_sport',
# 'TCP_dport',
# 'UDP_sport',
# 'UDP_dport',
 'pck_size',
 'Ether_type',
 'LLC_ssap',
 'LLC_ctrl',
 'EAPOL_version',
 'EAPOL_type',
 'EAPOL_len',
 'IP_version',
 'IP_ihl',
 'IP_tos',
 'IP_len',
 'IP_flags',
 'IP_DF',
 'IP_ttl',
 'IP_proto',
 'IP_options',
 'ICMP_type',
 'ICMP_code',
 'ICMP_seq',
 'TCP_dataofs',
 'TCP_flags',
 'TCP_FIN',
 'TCP_SYN',
 'TCP_RST',
 'TCP_PSH',
 'TCP_ACK',
 'TCP_window',
 'TCP_options',
 'UDP_len',
 'DHCP_options',
 'BOOTP_op',
 'BOOTP_htype',
 'BOOTP_hlen',
 'BOOTP_secs',
 'BOOTP_flags',
 'BOOTP_sname',
 'BOOTP_file',
 'BOOTP_options',
 'DNS_qr',
 'DNS_aa',
 'DNS_rd',
 'DNS_ra',
 'DNS_rcode',
 'DNS_qdcount',
 'DNS_ancount',
 'DNS_nscount',
 'DNS_arcount',
 'payload_bytes',    
 'entropy',
 'Protocol',
#"MAC",
 'Label']

train='./UNSW_BIG_train_IoTDevID.csv'
final="UNSW_test_IoTDevID.csv"
df = pd.read_csv(final,usecols=cols)#,header=None )
X_test =df[df.columns[0:-1]]
#X_test=np.array(X_test)
df[df.columns[-1]] = df[df.columns[-1]].astype('category')
y_test=df[df.columns[-1]].cat.codes  

df = pd.read_csv(train,usecols=cols)#,header=None )
X_train =df[df.columns[0:-1]]
#X_train=np.array(X_train)
df[df.columns[-1]] = df[df.columns[-1]].astype('category')
y_train=df[df.columns[-1]].cat.codes  




X_train1_best = X_train[best_collector]
clf_best = model.fit(X_train1_best, y_train)
X_test1 = X_test[best_collector]
y_pred = clf_best.predict(X_test1)
accuracy = (y_test.tolist()==y_pred).sum() / y_test.count()
print(accuracy)
print("F1: ", f1_score(y_test.tolist(),y_pred, average='macro'))



MEAN                           STD                           
0.7915310800134984             0.004871076131636332          
0.8194154636687533
F1:  0.7888234759184499
